# feature engineering
Start:
* one database with two collections that hold all the sprints and issues with all the raw data
* only addition to this raw data is a field which repository they belong to

End:

* Sprints:
  * Make aggregation of each sprint's metadata:
    * sprint planned duration in days
    * sprint overdue boolean (completedDate > endDate)
    * Number of issues inside sprint
    * Median number of issues with changed sprint
* Issues:
  * predictor: has sprint field in change log (boolean)
  * Number of sprint fields in change log (int)
  * Add originSprintId field (uuid)
  * Add above sprint metadata fields to every issue
  * All not used (custom)fields should be removed

In [2]:
# Auxiliary
import pymongo
import re
import json
import numpy as np
from datetime import datetime
from dateutil import parser

client = pymongo.MongoClient("mongodb://localhost:27017")
db = client['JiraRepos']

In [4]:
# Sprints
coll = db['02_Sprints']
sprints = coll.find()

# Iterate over each sprint
for sprint in sprints:
    # print(sprint)
    
    # sprint planned duration in days
    try:
        startDate = parser.parse(sprint['startDate'])
        endDate = parser.parse(sprint['endDate'])
        plannedDuration =  (endDate - startDate).days
    except:
        plannedDuration = np.nan

    # sprint overdue boolean (completedDate > endDate)
    try:
        completeDate = parser.parse(sprint['completeDate'])
        if (completeDate - endDate).days > 1:
            isOverdue = 1
        else:
            isOverdue = 0
    except:
        isOverdue = np.nan


    # Number of issues inside sprint
    noOfIssues = len(sprint['issues'])

    coll.update_one({'id': sprint['id']}, {'$set': {'plannedDuration': plannedDuration, 'isOverdue': isOverdue, 'noOfIssues': noOfIssues}})
    # print(sprint['id'], plannedDuration, isOverdue, noOfIssues)



In [ ]:
def get_origin_sprint(issue_id):
    sprint_id = re.search(r'id=(.*?),', issue_id[]).group(1)

In [ ]:
# Issues
coll = db['01_Issues']
issues = coll.find()

# Number of sprint fields in change log (int)
for issue in issues:
    sprint_changes = 0
    print(issue['id'])
    print(issue['changelog']['total'])
    histories = issue['changelog']['histories']
    for history in histories:
        for item in history['items']:
            if 'sprint' in item['field'].lower():
                sprint_changes += 1
    
    print("sprint changes: " + str(sprint_changes))

    # Add above sprint metadata fields to every issue
    # All not used (custom)fields should be removed

In [ ]:
# Sprints that need the new Issue data
# Median number of issues with changed sprint